# Calculating Atomic Contributions for Molecules Based on Graph Convolutional  QSAR Model. A tutorial. 
<img src="atomic_contributions_tutorial_data/index.png">

## Why do the DeepChem Tutorial?
###  You will learn   to find atoms of molecules important for the activity, and visualize them. We will learn on two tasks: classification and regression.
###  Contributions are also known as "attributions" , "coloration" etc. in literature. Atomic contributions are  defined as the difference in activity of the whole molecule and fragment remaining after atom removal. This is one of model interpretation methods [1], analogous to Similarity maps [2] in QSAR domain, or occlusion methods in other fields (image classification etc).



#### Mariia Matveieva, Pavel Polishchuk. Institute of Molecular and Translational Medicine, Palacky University, Olomouc, Czech Republic.


In [1]:
import pandas as pd
import deepchem as dc
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw, PyMol, rdFMCS
from rdkit.Chem.Draw import IPythonConsole
from rdkit import rdBase
from deepchem import metrics
from IPython.display import Image, display
from rdkit.Chem.Draw import SimilarityMaps
import tensorflow as tf

## 1. First, let's build a classification QSAR model for Blood-brain barrier permeability
### BBB permeability is the ability of compounds to  enter  central nervous system. Here we use dataset of relatively small  compounds which are transported by diffusion without any  carriers.  The property is defined as log10 (concentration in brain / concentration in blood). Compounds with positive  value (and 0) are labeled active, others inactive.  After modelling we will identify atoms favorable and unfavorable for diffusion.

In [2]:
# 1. Create dataset

In [3]:
DATASET_FILE ='atomic_contributions_tutorial_data/logBB.sdf'

In [4]:
# create RDKit mol objects, we will need them later
mols = [m for m in Chem.SDMolSupplier(DATASET_FILE) if m is not None ]

In [5]:
len(mols)

298

In [6]:
loader = dc.data.SDFLoader(tasks=["logBB_class"], 

                           featurizer=dc.feat.ConvMolFeaturizer(),
                           sanitize = True)

dataset = loader.create_dataset(DATASET_FILE, shard_size=2000)



In [7]:
dataset.ids.shape

(298,)

In [8]:
# 2. Build a model

In [9]:
np.random.seed(2020)
tf.random.set_seed(2020)

In [10]:
m = dc.models.GraphConvModel(1, mode="classification", nb_epoch=10,batch_normalize=False, restore=False, batch_size = 100)
m.fit(dataset)


/home/maria/anaconda3/envs/deepchem_testing/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.5348200480143229

In [11]:
# 3. Test set performance

In [12]:
TEST_DATASET_FILE = 'atomic_contributions_tutorial_data/logBB_test_.sdf'
loader = dc.data.SDFLoader(tasks=["p_np"], sanitize= True,
                           featurizer=dc.feat.ConvMolFeaturizer())

test_dataset = loader.create_dataset(TEST_DATASET_FILE, shard_size=2000)



In [13]:
pred =  m.predict(test_dataset)

In [14]:
pred = np.argmax(np.squeeze(pred),axis= 1)

In [15]:
ba = metrics.balanced_accuracy_score(y_true=test_dataset.y, y_pred=pred)

In [16]:
# balanced accuracy is high enough to procced to model interpretation - i.e. atomic contributions
ba 

0.7444444444444445

## 2. Now, let's prepare dataset of fragments based on training set (any other unseen data set of interest can be used). These fragments will be used to evaluate contributions of atoms.
### For each molecule we will generate a list of ConvMol objects: featurizations for single atom-depleted versions of the  molecule (iterate over all heavy atoms in the molecule)
### All we need is to load again the training set,  but when featurizing it,  set
##### per_atom_fragmentation=True

In [17]:
loader = dc.data.SDFLoader(tasks=[],# dont need task (moreover, passing the task can lead to inconsitencies in data shapes)
                        featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=True),
                        sanitize=True) 

frag_dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)

/home/maria/anaconda3/envs/deepchem_testing/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/mnt/d/deepchem/deepchem/data/data_loader.py:919: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(features), valid_inds


In [18]:
# dataset has the same number of  rows  than original training set, 
# but each row is a fragment list of corresponding molecule. 
# Molecules go in the same order as   original input file. 
# IMPORTANT: fragments order depends on input format: if sdf, then fragment order is the same, as atom order in corresponding mol blocks
# if SMILES (i.e. csv with molecules represented as SMILES), then order is given by RDKit CanonicalRankAtoms

frag_dataset.X.shape


(298,)

In [19]:
frag_dataset.ids.shape# ids indicate name of the molecule which  fragments ocome from

(298,)

In [20]:
# transform dataset to flatten fragments lists to make it suitable for prediction 

In [21]:
tr = dc.trans.FlatteningTransformer(frag_dataset)

In [22]:
frag_dataset = tr.transform(frag_dataset)

In [23]:
# dataset has now number of  rows equal total number of fragments in all molecules (i.e. flat)
frag_dataset.X.shape


(5111,)

In [24]:
# the ids are expanded so that each fragment has an id of parent molecule
frag_dataset.ids.shape

(5111,)

## 3. Now we will predict the activity for  molecules and for their fragments. Then, for each fragment,  we'll find the activity difference - that is atomic contribution of removed atom, corresponding to that fragment.
### Note: here, in classification context,  we use probabilistic output of the model as the activity. So, contribution is probability difference, i.e. "how much a given atom increases/decreases the probability of the molecule to be active"

In [25]:
# whole  molecules
pred = np.squeeze(m.predict(dataset))[:, 1] # probabilitiy of class 1
pred = pd.DataFrame(pred, index=dataset.ids, columns=["Molecule"])  # turn to dataframe for convinience

In [26]:
# fragments
pred_frags =np.squeeze(m.predict(frag_dataset))[:, 1]
pred_frags = pd.DataFrame(pred_frags, index=frag_dataset.ids, columns=["Fragment"])

# 4.  Let's finally find atomic contributions:

In [27]:
# merge 2 dataframes by molecule names
df = pd.merge(pred_frags, pred, right_index=True, left_index=True)
# find contribs
df['Contrib'] = df["Molecule"] - df["Fragment"]

In [28]:
df

,Fragment,Molecule,Contrib
C#CC1(O)CCC2C3C(C)CC4=C(CCC(=O)C4)C3CCC21C,0.756531,0.811546,0.055015
C#CC1(O)CCC2C3C(C)CC4=C(CCC(=O)C4)C3CCC21C,0.752750,0.811546,0.058796
C#CC1(O)CCC2C3C(C)CC4=C(CCC(=O)C4)C3CCC21C,0.747007,0.811546,0.064539
C#CC1(O)CCC2C3C(C)CC4=C(CCC(=O)C4)C3CCC21C,0.815875,0.811546,-0.004329
C#CC1(O)CCC2C3C(C)CC4=C(CCC(=O)C4)C3CCC21C,0.741801,0.811546,0.069745
...,...,...,...
c1cncc(C2CCCN2)c1,0.780478,0.813036,0.032558
c1cncc(C2CCCN2)c1,0.722650,0.813036,0.090386
c1cncc(C2CCCN2)c1,0.721609,0.813036,0.091427
c1cncc(C2CCCN2)c1,0.683302,0.813036,0.129734


#  5. Visualize contributions with RDKit Similarity Maps

In [29]:
def vis_contribs(mols, df, smi_or_sdf = "sdf"): 
    # input format of file, which was used to create dataset  determines the order of atoms, 
    # so we take it into account for correct mapping!
    maps = []
    for mol  in mols:
        wt = {}
        if smi_or_sdf == "smi":
            for n,atom in enumerate(Chem.rdmolfiles.CanonicalRankAtoms(mol)):
                wt[atom] = df.loc[mol.GetProp("_Name"),"Contrib"][n]
        if smi_or_sdf == "sdf":        
            for n,atom in enumerate(range(mol.GetNumHeavyAtoms())):
                wt[atom] = df.loc[Chem.MolToSmiles(mol),"Contrib"][n]
        maps.append(SimilarityMaps.GetSimilarityMapFromWeights(mol,wt))
    return maps    

### Let's look at some pictures:

In [ ]:
np.random.seed(2000)
maps = vis_contribs(np.random.choice(np.array(mols),20), df)

### We can see, that aromatics or aliphatics  have positive impact on membrane permeability, while polar or charged heteroatoms  have negative influence. This is generally consistent with literature data.

## 6. Let's take  a regression task, aquatic toxicity (towards water organism T. pyriformis).
### Toxicity is defined as log10 (IGC50) (concentration that inhibits colony growth by 50 %). Toxicophores for T. pyriformis will be identified by atomic contributions.
### All the above steps are the same: load data, featurize, build a model, create dataset of fragments, find contributions and visualize them
### Note: this time as it is regression, contributions will be in activity units, not probability


In [31]:
DATASET_FILE ='atomic_contributions_tutorial_data/Tetrahymena_pyriformis_Work_set_OCHEM.sdf'

In [32]:
# create RDKit mol objects, we will need them later
mols = [m for m in Chem.SDMolSupplier(DATASET_FILE) if m is not None ]

In [33]:
len(mols)

1424

In [34]:
loader = dc.data.SDFLoader(tasks=["IGC50"], 
                           featurizer=dc.feat.ConvMolFeaturizer(), sanitize = True)

dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)

In [35]:
np.histogram(dataset.y)

(array([ 16,  79, 156, 233, 320, 275, 185, 104,  45,  11]),
 array([0.49, 1.077, 1.664, 2.251, 2.838, 3.425, 4.012, 4.599, 5.186,
        5.773, 6.36], dtype=object))

In [36]:
np.random.seed(2020)
tf.random.set_seed(2020)
m = dc.models.GraphConvModel(1, mode="regression", nb_epoch=40,batch_normalize=False, restore=False)
m.fit(dataset)

/home/maria/anaconda3/envs/deepchem_testing/lib/python3.7/site-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


0.3669636917114258

In [37]:
TEST_DATASET_FILE = 'atomic_contributions_tutorial_data/Tetrahymena_pyriformis_Test_set_OCHEM.sdf'
loader = dc.data.SDFLoader(tasks=["IGC50"], sanitize= True,
        
                           featurizer=dc.feat.ConvMolFeaturizer())

test_dataset = loader.create_dataset(TEST_DATASET_FILE, shard_size=2000)


In [38]:
pred = m.predict(test_dataset)

In [39]:
mse=metrics.mean_squared_error(y_true=test_dataset.y, y_pred=pred)
r2=metrics.r2_score(y_true=test_dataset.y, y_pred=pred)

In [40]:
mse

0.41432252518295415

In [41]:
r2 # r2 is not very high, but acceptable

0.6248392117899626

In [42]:
# load again the training set, 
# but when featurizing it,  set per_atom_fragmentation=True

loader = dc.data.SDFLoader(tasks=[], # dont need any task
                           sanitize = True,
                           featurizer=dc.feat.ConvMolFeaturizer(per_atom_fragmentation=True)) 

frag_dataset = loader.create_dataset(DATASET_FILE, shard_size=5000)

/home/maria/anaconda3/envs/deepchem_testing/lib/python3.7/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/mnt/d/deepchem/deepchem/data/data_loader.py:919: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(features), valid_inds


In [43]:
tr = dc.trans.FlatteningTransformer(frag_dataset) # flatten dataset and add ids to each fragment

In [44]:
frag_dataset = tr.transform(frag_dataset)

In [45]:
# whole molecules
pred = m.predict(dataset)
pred = pd.DataFrame(pred, index=dataset.ids, columns=["Molecule"])  # turn to dataframe for convinience

In [46]:
# fragments
pred_frags = m.predict(frag_dataset)
pred_frags = pd.DataFrame(pred_frags, index=frag_dataset.ids, columns=["Fragment"])  # turn to dataframe for convinience

In [47]:
# merge 2 dataframes by molecule names
df = pd.merge(pred_frags, pred, right_index=True, left_index=True)
# find contribs
df['Contrib'] = df["Molecule"] - df["Fragment"]

In [ ]:
# Lets take some molecules with moderate activity (not extremely active/inactive)

In [ ]:
maps = vis_contribs([mol for mol in mols if float(mol.GetProp("IGC50"))>3 and float(mol.GetProp("IGC50"))<4], df)

### We can see, that known toxicophores are in green, namely, nitro-aromatics, halo-aromatics, long alkyl chains, aldehyde; while carboxylic group, alcohol, amino are detoxyfying, as is consistent with literature [3]

# Appendix

## In this tutorial we operated on sdf files. However,  If we use csv with SMILES as input , the order of atoms in the dataframe DOES NOT correspond to original atom order. If we  want to recover the original atom order for each molecule (to have it in our main dataframe), we need to use RDKit's  Chem.rdmolfiles.CanonicalRankAtoms. Here are some utils to do this.
## We can add a column with atom ids (as in input molecules) and use the resulting dataframe for analysis with any other  software, outside "python-rdkit-deepchem" environment

In [ ]:
def get_mapping(mols, mol_names): 
    """perform mapping:
    atom number original <-> atom number(position) 
    after ranking (both 1-based)"""
    # mols - RDKit mols
    # names  - any seq of strings
    # return list of nested lists: [[molecule, [atom , atom, ..], [...]]
    assert(len(mols)==len(mol_names))
    mapping = []
    for m,n in zip(mols, mol_names):
        atom_ids = [i+1 for i in list(
            Chem.rdmolfiles.CanonicalRankAtoms(m))]
        mapping.append([n, atom_ids
                   ])
    
    return mapping
    

In [ ]:
def append_atomid_col(df, mapping):
    # add column with CORRECT atom number(position)
    for i in mapping:
        df.loc[i[0],"AtomID"] = i[1]
    return df   

# Bibliography:

###### 1. Polishchuk, P., O. Tinkov, T. Khristova, L. Ognichenko, A. Kosinskaya, A. Varnek & V. Kuz’min (2016) Structural and Physico-Chemical Interpretation (SPCI) of QSAR Models and Its Comparison with Matched Molecular Pair Analysis. Journal of Chemical Information and Modeling, 56, 1455-1469.
###### 2. Riniker, S. & G. Landrum (2013) Similarity maps - a visualization strategy for molecular fingerprints and machine-learning methods. Journal of Cheminformatics, 5, 43.
###### 3. M. Matveieva, M. T. D. Cronin, P. Polishchuk, Mol. Inf. 2019, 38, 1800084. 